# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [14]:
# First, we have to encode the homeowner column, and the loan_status column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])

df.head()

# Create our features
x_cols = [i for i in df.columns if i not in "loan_status"]
X = df[x_cols]

# Create our target
y = df["loan_status"]

In [15]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [17]:
# Check the balance of our target values
y.value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [18]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [19]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [20]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [50]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [51]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [52]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.988957164943355

In [53]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  579,    10],
       [   96, 18699]], dtype=int64)

In [54]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.98      0.99      0.92      0.99      0.98       589
          1       1.00      0.99      0.98      1.00      0.99      0.98     18795

avg / total       1.00      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [84]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({1: 56241, 0: 56241})

In [85]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_resampled, y_resampled)

LogisticRegression()

In [86]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9947132202465074

In [87]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  586,     3],
       [  103, 18692]], dtype=int64)

In [88]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.92      0.99      0.99       589
          1       1.00      0.99      0.99      1.00      0.99      0.99     18795

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [89]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_smote, y_smote= SMOTE(sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
Counter(y_smote)

Counter({1: 56241, 0: 56241})

In [90]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_smote, y_smote)

LogisticRegression()

In [92]:
# Calculated the balanced accuracy score
y_smote_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_smote_pred)

0.9947398230664063

In [93]:
# Display the confusion matrix
confusion_matrix(y_test, y_smote_pred)

array([[  586,     3],
       [  102, 18693]], dtype=int64)

In [94]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.92      0.99      0.99       589
          1       1.00      0.99      0.99      1.00      0.99      0.99     18795

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [71]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids()
X_cc_resampled, y_cc_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_cc_resampled)

Counter({0: 1911, 1: 1911})

Counter({0: 1911, 1: 1911})

In [72]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_cc_resampled, y_cc_resampled)

LogisticRegression()

In [76]:
# Calculate the balanced accuracy score
y_cc_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_cc_pred)

0.9940239407312659

In [77]:
# Display the confusion matrix
confusion_matrix(y_test, y_cc_pred)

array([[  585,     4],
       [   97, 18698]], dtype=int64)

In [78]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_cc_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.99      0.99      0.92      0.99      0.99       589
          1       1.00      0.99      0.99      1.00      0.99      0.99     18795

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [95]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN()
X_smoteen_resampled, y_smoteen_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_smoteen_resampled)

Counter({0: 55672, 1: 55867})

In [96]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_smoteen_resampled, y_smoteen_resampled)

LogisticRegression()

In [97]:
# Calculate the balanced accuracy score
y_smoteen_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_smoteen_pred)

0.9947664258863052

In [98]:
# Display the confusion matrix
confusion_matrix(y_test, y_smoteen_pred)

array([[  586,     3],
       [  101, 18694]], dtype=int64)

In [99]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smoteen_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.92      0.99      0.99       589
          1       1.00      0.99      0.99      1.00      0.99      0.99     18795

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   THE SMOTEEN combined over/under sampling had the best balanced accuracy at 0.9947664258863052, though all were pretty similar

2. Which model had the best recall score?

    All the models had high recall, even the simple logistic regression.  All the imbalanced models were slightly higher, showing 0.99 recall for both positives and negatives.
    

3. Which model had the best geometric mean score?

    All models showed the same F1 and geometric mean scores, with 0.99 respectively for all models.
